In [1]:
from torchrecsys.datasets import InteractionsDataset, SequenceDataset
from torchrecsys.models import BaseModel
from torchrecsys.task import Ranking
from torchrecsys.layers import BruteForceLayer
from torchrecsys.utils import feature_catalog
import pandas as pd

In [2]:
ratings = pd.read_csv("ml-100k/ratings.csv").rename(columns={"userId": "user", "movieId": "item"})
movies = pd.read_csv("ml-100k/movies.csv").rename(columns={"movieId": "item"})
users = ratings[["user"]].drop_duplicates()
movies['title'] = movies.title.astype('category')
movies['genres'] = movies.genres.astype('category')

In [3]:
dataset = InteractionsDataset(ratings,movies, users)

In [4]:
dataset.item_features

{1: array([1, 'Toy Story (1995)',
        'Adventure|Animation|Children|Comedy|Fantasy'], dtype=object),
 2: array([2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'], dtype=object),
 3: array([3, 'Grumpier Old Men (1995)', 'Comedy|Romance'], dtype=object),
 4: array([4, 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'],
       dtype=object),
 5: array([5, 'Father of the Bride Part II (1995)', 'Comedy'], dtype=object),
 6: array([6, 'Heat (1995)', 'Action|Crime|Thriller'], dtype=object),
 7: array([7, 'Sabrina (1995)', 'Comedy|Romance'], dtype=object),
 8: array([8, 'Tom and Huck (1995)', 'Adventure|Children'], dtype=object),
 9: array([9, 'Sudden Death (1995)', 'Action'], dtype=object),
 10: array([10, 'GoldenEye (1995)', 'Action|Adventure|Thriller'], dtype=object),
 11: array([11, 'American President, The (1995)', 'Comedy|Drama|Romance'],
       dtype=object),
 12: array([12, 'Dracula: Dead and Loving It (1995)', 'Comedy|Horror'],
       dtype=object),
 13: array([13, 'Balto (1995)

In [5]:
dataset[1]

array([1.00000000e+00, 3.00000000e+00, 4.00000000e+00, 9.64981247e+08])

In [6]:
movies

,item,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
movie_catalog = feature_catalog(movies, id_column_name="movieId")

KeyError: Index(['movieId'], dtype='object')

In [ ]:
movie_catalog[1]["categorical"]

In [ ]:
movies.columns.values.tolist().index("title")